# KIT Datenauswertung

In [73]:
import pandas as pd
import numpy as np

In [74]:
path = "Abschiednahme.csv"
df = pd.read_csv(path, sep =';', encoding="cp1252")
   

In [75]:
df

,Nr,ID,Indikation,Geschlecht,Alter,Bezug,nicht mgl,"mgl, nicht erf,",erfolgt,profitiert,kooperativ,freundlich,ablehnend,aggressiv,klammernd,fordernd,vorsichtig,gleichgültig,RM
0,1,1.1,0.0,0,NaN,2.0,"1,2",NaN,NaN,NaN,4.0,NaN,0.0,0.0,1.0,2.0,0.0,0.0,1
1,2,1.2,0.0,0,76.0,0.0,"1,2",NaN,NaN,NaN,4.0,NaN,1.0,0.0,0.0,0.0,0.0,4.0,1
2,3,2.0,1.0,1,77.0,1.0,NaN,NaN,0,0.0,5.0,NaN,0.0,0.0,2.0,0.0,0.0,0.0,1
3,4,3.0,NaN,1,58.0,4.0,NaN,NaN,0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,5,4.0,0.0,0,30.0,NaN,1,NaN,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN,"0,4"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,237,149.1,NaN,0,55.0,5.0,0,NaN,NaN,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
236,238,149.2,NaN,0,25.0,10.0,0,NaN,NaN,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,239,150.0,0.0,0,56.0,4.0,2,NaN,NaN,1.0,5.0,NaN,0.0,0.0,3.0,0.0,NaN,0.0,4
238,240,151.1,0.0,0,32.0,2.0,NaN,0.0,NaN,1.0,5.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0


In [78]:
def parser(flo):
    r_array = flo
    if ~np.isnan(flo):
        r_array= [i for i in str(flo).replace(".","")]
        
    return r_array

In [84]:
df["nicht mgl"] = df.apply(lambda x: parser(x["nicht mgl"]),axis=1)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [83]:
df

,Nr,ID,Indikation,Geschlecht,Alter,Bezug,nicht mgl,"mgl, nicht erf,",erfolgt,profitiert,kooperativ,freundlich,ablehnend,aggressiv,klammernd,fordernd,vorsichtig,gleichgültig,RM
0,1,1.1,"[0, 0]",0,NaN,2.0,"1,2",NaN,NaN,NaN,4.0,NaN,0.0,0.0,1.0,2.0,0.0,0.0,1
1,2,1.2,"[0, 0]",0,76.0,0.0,"1,2",NaN,NaN,NaN,4.0,NaN,1.0,0.0,0.0,0.0,0.0,4.0,1
2,3,2.0,"[1, 0]",1,77.0,1.0,NaN,NaN,0,0.0,5.0,NaN,0.0,0.0,2.0,0.0,0.0,0.0,1
3,4,3.0,NaN,1,58.0,4.0,NaN,NaN,0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,5,4.0,"[0, 0]",0,30.0,NaN,1,NaN,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN,"0,4"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,237,149.1,NaN,0,55.0,5.0,0,NaN,NaN,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
236,238,149.2,NaN,0,25.0,10.0,0,NaN,NaN,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,239,150.0,"[0, 0]",0,56.0,4.0,2,NaN,NaN,1.0,5.0,NaN,0.0,0.0,3.0,0.0,NaN,0.0,4
238,240,151.1,"[0, 0]",0,32.0,2.0,NaN,0.0,NaN,1.0,5.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0
